In [20]:
import pandas as pd
from sklearn import preprocessing
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

c:\Users\kuils\anaconda3\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

## **대출기간**

In [3]:
train_data["대출기간"].value_counts()
# train["범죄발생지"].value_counts().values

data_frequency = {' 36 months':2, ' 60 months':1}
data_frequency.get("대출기간")

In [4]:
#train
data_frequency.get(train_data.loc[1, "대출기간"])  # '차도':13

for loc in range(0, train_data.shape[0]):
    encd = data_frequency.get(train_data.loc[loc, "대출기간"])
    train_data.loc[loc, "대출기간(encode)"] = encd
train_data = train_data.drop(columns='대출기간')
train_data.head(2)     # 84406, 27

#test
data_frequency.get(test_data.loc[1, "대출기간"])  # '차도':13

for loc in range(0, test_data.shape[0]):
    encd = data_frequency.get(test_data.loc[loc, "대출기간"])
    test_data.loc[loc, "대출기간(encode)"] = encd
test_data = test_data.drop(columns='대출기간')
test_data.head(2)     # 84406, 27

,ID,대출금액,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간(encode)
0,TEST_00000,16800000,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0,2.0
1,TEST_00001,8400000,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0,2.0


## **주택소유상태**

In [5]:
data_frequency = {'MORTGAGE':4, 'RENT':3, 'OWN':2,'ANY':1}
data_frequency.get("주택소유상태")

In [6]:
#train
data_frequency.get(train_data.loc[1, "주택소유상태"])  # '차도':13

for loc in range(0, train_data.shape[0]):
    encd = data_frequency.get(train_data.loc[loc, "주택소유상태"])
    train_data.loc[loc, "주택소유상태(encode)"] = encd
train_data = train_data.drop(columns='주택소유상태')
train_data.head(2)     # 84406, 27

#test
data_frequency.get(test_data.loc[1, "주택소유상태"])  # '차도':13

for loc in range(0, test_data.shape[0]):
    encd = data_frequency.get(test_data.loc[loc, "주택소유상태"])
    test_data.loc[loc, "주택소유상태(encode)"] = encd
test_data = test_data.drop(columns='주택소유상태')
test_data.head(2)     # 84406, 27

,ID,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간(encode),주택소유상태(encode)
0,TEST_00000,16800000,8 years,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0,2.0,4.0
1,TEST_00001,8400000,5 years,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0,2.0,3.0


## **대출목적**

In [7]:
data_frequency = {'부채 통합':12, '신용 카드':11, '주택 개선':10,'기타':9, '주요 구매':8, '의료':7, '자동차':6, '소규모 사업':5\
                  ,'이사':4,'휴가':3, '주택':2,'재생 에너지':1}
data_frequency.get("대출목적")

In [8]:
#train
data_frequency.get(train_data.loc[1, "대출목적"])  

for loc in range(0, train_data.shape[0]):
    encd = data_frequency.get(train_data.loc[loc, "대출목적"])
    train_data.loc[loc, "대출목적(encode)"] = encd
train_data = train_data.drop(columns='대출목적')
train_data.head(2)     

#test
data_frequency.get(test_data.loc[1, "대출목적"])  

for loc in range(0, test_data.shape[0]):
    encd = data_frequency.get(test_data.loc[loc, "대출목적"])
    test_data.loc[loc, "대출목적(encode)"] = encd
test_data = test_data.drop(columns='대출목적')
test_data.head(2)     

,ID,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간(encode),주택소유상태(encode),대출목적(encode)
0,TEST_00000,16800000,8 years,132000000,19.64,12,0,394692,146604.0,0.0,0.0,2.0,4.0,10.0
1,TEST_00001,8400000,5 years,89971200,15.84,25,0,0,0.0,0.0,0.0,2.0,3.0,12.0


## **대출등급**

In [10]:
label_encoder_target = preprocessing.LabelEncoder() 
train_data['대출등급'] = label_encoder_target.fit_transform(train_data['대출등급'] )

## **근로기간**

In [12]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)
test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

## **x,y 분리 ,  표준편차 scailing**

scail 했을때, 더 안좋아짐...

In [21]:
train_data_feature = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data_feature = train_data_feature.drop(columns='대출등급')
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# columns_to_normalize = ['연간소득', '총계좌수', '부채_대비_소득_비율','대출금액', '총상환원금', '총상환이자', '총연체금액']
# train_data_feature[columns_to_normalize] = scaler.fit_transform(train_data_feature[columns_to_normalize])
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data_feature,train_data_target,test_size=0.2, shuffle=True,random_state=34)

In [65]:
# scaler = StandardScaler()
# columns_to_normalize = ['연간소득', '총계좌수', '부채_대비_소득_비율','대출금액', '총상환원금', '총상환이자', '총연체금액']
# test_data[columns_to_normalize] = scaler.fit_transform(test_data[columns_to_normalize])


## **over sampling (SMOTE)**

In [55]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=19)
train_data_x_smote, train_data_y_smote = smote.fit_resample(train_data_x, train_data_y)
print(train_data_x_smote.shape, train_data_y_smote.shape)

(161399, 13) (161399,)


In [80]:
smote = SMOTE(random_state=19)
train_data_feature_smote, train_data_target_smote = smote.fit_resample(train_data_feature, train_data_target)


## **train**

In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1)
    }

    model = XGBClassifier(**param, random_state=1004)
    model.fit(train_data_x_smote, train_data_y_smote)
    preds = model.predict(valid_data_x)
    accuracy = accuracy_score(valid_data_y, preds)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # 예를 들어 100번의 시도

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [57]:
optuna.visualization.plot_param_importances(study) # 파라미터 중요도 확인 그래프
optuna.visualization.plot_optimization_history(study) # 최적화 과정 시각화

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.8550807414715198, 0.8536268757464043, 0.8595461861986604,
                    0.8433459681188016, 0.8427748065839348, 0.8476556415182512,
                    0.8571576925073991, 0.8497845163300275, 0.8572096162832962,
                    0.8562230645412534, 0.8525364764525676, 0.8528480191079495,
                    0.8584038631289267, 0.8536268757464043, 0.8576769302663689,
                    0.8553922841269017, 0.8491095072433668, 0.8581442442494418,
                    0.8581961680253388, 0.8527960953320526, 0.8579884729217508,
                    0.8596500337504543, 0.8546134274884469, 0.8597019575263514,
                    0.8592346435432785, 0.8537826470740952, 0.8491614310192637,
                    0.8583519393530298, 0.854665351264344, 0.8570019211797082]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.8550807414715198, 0.8550807414715198, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8595461861986604, 0.8595461861986604, 0.8595461861986604,
                    0.8596500337504543, 0.8596500337504543, 0.8597019575263514,
                    0.8597019575263514, 0.8597019575263514, 0.8597019575263514,
                    0.8597019575263514, 0.8597019575263514, 0.8597019575263514]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

## **predict**

In [68]:
model = XGBClassifier(**study.best_trial.params)
model.fit(train_data_x, train_data_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5274517494810426, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2.4914336579565317,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06449015444332712,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2243, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
model = XGBClassifier(n_estimators= 5321, max_depth= 14, min_child_weight= 6, gamma= 1.4483766944718863, learning_rate= 0.04401771495020846, colsample_bytree= 0.8301747712346244, reg_lambda= 0.0722023820701971, reg_alpha= 0.22100718915407028, subsample= 0.8746729153116067)
model.fit(train_data_feature_smote, train_data_target_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8301747712346244, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=1.4483766944718863,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04401771495020846,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5321, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [82]:
# test_data = test_data.drop(columns='ID')
test_pred = model.predict(test_data)

In [85]:
test_pred_decoded = label_encoder_target.inverse_transform(test_pred)
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred_decoded
submission_data.to_csv('./xgboost.csv', index=False)

scailing, log scailing, onehotencoding

In [19]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
대출금액,96294.0,1.830400e+07,1.032908e+07,1200000.0,10200000.00,16800000.00,2.400000e+07,4.200000e+07
근로기간,96294.0,5.738551e+00,3.768899e+00,0.0,2.00,6.00,1.000000e+01,1.000000e+01
연간소득,96294.0,9.392672e+07,9.956871e+07,0.0,57600000.00,78000000.00,1.128000e+08,1.080000e+10
부채_대비_소득_비율,96294.0,1.937959e+01,3.356956e+01,0.0,12.65,18.74,2.554000e+01,9.999000e+03
총계좌수,96294.0,2.530483e+01,1.208857e+01,4.0,17.00,24.00,3.200000e+01,1.690000e+02
최근_2년간_연체_횟수,96294.0,3.456809e-01,9.191187e-01,0.0,0.00,0.00,0.000000e+00,3.000000e+01
총상환원금,96294.0,8.225035e+05,1.027745e+06,0.0,307572.00,597696.00,1.055076e+06,4.195594e+07
총상환이자,96294.0,4.282282e+05,4.402111e+05,0.0,134616.00,287004.00,5.702160e+05,5.653416e+06
총연체금액,96294.0,5.438058e+01,1.414769e+03,0.0,0.00,0.00,0.000000e+00,7.576800e+04
연체계좌수,96294.0,5.805138e-03,7.996622e-02,0.0,0.00,0.00,0.000000e+00,4.000000e+00
